# 目标：

    1、利用resnet-50构建模型，并开始训练

# 读取数据

    1、读取后缀为.h5的模型。
    2、并区分训练集和测试集。

In [10]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [136]:
TRAIN_DATA_PATH='./data/resnet50_dataset/train_signs.h5'
TEST_DATA_PATH='./data/resnet50_dataset/test_signs.h5'
LOG_DIR = './logs/'
num_classes = 6


In [18]:
def data_loader(path,is_training=True):
    
    f = h5py.File(path,'r')
    if is_training:
        set_x = f['train_set_x'][:]
        set_y = f['train_set_y'][:]
    else:
        set_x = f['test_set_x'][:]
        set_y = f['test_set_y'][:]
    
    # data的尺寸，转换成resnet需要的尺寸
    set_x = np.asarray([ cv2.resize(pic,(224,224)) for pic in set_x])
    # label_one_hot
    def to_one_hot(y,num_classes):
        return np.eye(num_classes)[y.reshape(-1)]    
    set_y = to_one_hot(set_y,num_classes)
    

    
    
    return set_x,set_y

In [19]:
train_data_x,train_data_y = data_loader(TRAIN_DATA_PATH,is_training=True)
test_data_x,test_data_y = data_loader(TEST_DATA_PATH,is_training=False)

In [22]:
train_data_x.shape,train_data_y.shape

((1080, 224, 224, 3), (1080, 6))

# 构建模型

    1、尝试利用tf.slim来构建模型。
    2、构建完毕后使用summary来查看

In [33]:
import tensorflow.contrib.slim as slim
import tensorflow as tf

In [79]:
def my_block(inputs,output_dim_list,stride):
    # 如果输入输出的channel不同
    input_channel=inputs.get_shape()[-1]
    
    if output_dim_list[2] == input_channel:
        print('hehe')
        short_cuts = slim.max_pool2d(inputs,(1,1),stride)
    else:
        print('haha')
        # 这里只是为了提升channel而已
        short_cuts = slim.conv2d(inputs,output_dim_list[2],(1,1),stride,padding='SAME')
        short_cuts = slim.batch_norm(short_cuts)
        
    # 第一个conv2d的步长固定为1
    res_path = slim.conv2d(inputs,output_dim_list[0],(1,1),stride=1,padding='SAME')
    res_path = slim.batch_norm(res_path,activation_fn=tf.nn.relu)
    
    # 第二个conv2d的步长视情况而定
    res_path = slim.conv2d(res_path,output_dim_list[1],(3,3),stride,padding='SAME')
    res_path = slim.batch_norm(res_path,activation_fn=tf.nn.relu)
    
    # 第三个conv2d的步长还是1*1
    res_path = slim.conv2d(res_path,output_dim_list[2],(1,1),stride=1,padding='SAME')
    res_path = slim.batch_norm(res_path)
    

    res_output = res_path+short_cuts
    
    return tf.nn.relu(res_output)
    

In [152]:
tf.reset_default_graph()

x_input = tf.placeholder(shape=(None,224,224,3),
                         dtype=tf.float32,name='input_data')
y_input = tf.placeholder(shape=(None,6),
                         dtype=tf.float32,name='input_label')

with tf.variable_scope('resnet_50'):
    '''
        block_conv1
    '''
    
    with tf.variable_scope('Block_1'):
        net = slim.conv2d(x_input,64,(7,7),stride=2,padding='SAME')
        net = slim.max_pool2d(net,kernel_size=(3,3),stride=2,padding='SAME')
    
    '''
        block_conv2
    '''
    with tf.variable_scope('Block_2'):
        block_conv2_nums=3
        for i in range(block_conv2_nums):
            if i<block_conv2_nums-1:
                temp_stride = 1
            else:
                temp_stride = 2
            with tf.variable_scope('unit_%d'%(i+1)):
                net = my_block(net,[64,64,256],temp_stride)
    '''
        block_conv3
    '''
    with tf.variable_scope('Block_3'):
        block_conv3_nums=4
        for i in range(block_conv3_nums):
            if i<block_conv3_nums-1:
                temp_stride = 1
            else:
                temp_stride = 2
            with tf.variable_scope('unit_%d'%(i+1)):
                net = my_block(net,[128,128,512],temp_stride)
        
    '''
        block_conv4
    '''
    with tf.variable_scope('Block_4'):
        block_conv4_nums=6
        for i in range(block_conv4_nums):
            if i<block_conv4_nums-1:
                temp_stride = 1
            else:
                temp_stride = 2
            with tf.variable_scope('unit_%d'%(i+1)):
                net = my_block(net,[256,256,1024],temp_stride)
    '''
        block_conv5
    '''
    with tf.variable_scope('Block_5'):
        block_conv5_nums=3
        for i in range(block_conv5_nums):
            temp_stride=1
            with tf.variable_scope('unit_%d'%(i+1)):
                net = my_block(net,[512,512,2048],temp_stride)
    
    '''
        全局平均池化
    '''
    net = tf.reduce_mean(net,[1,2],keepdims=True)
    
    '''
        接下来有两种选择：
            1、用fc将平均池化结果变成我们想要的结果尺寸
            2、用卷积将平均池化结果变成我们想要的结果尺寸
            3、我们用卷积
    '''
    
    net = slim.conv2d(net,num_classes,(1,1),1)
    
    net = tf.reshape(net,(-1,6))
    
    net_output = slim.softmax(net)
    
    print(net_output)
    

haha
hehe
hehe
haha
hehe
hehe
hehe
haha
hehe
hehe
hehe
hehe
hehe
haha
hehe
hehe
Tensor("resnet_50/softmax/Reshape_1:0", shape=(?, 6), dtype=float32)


In [153]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=net,labels=y_input)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=3e-4).minimize(loss)

In [154]:
batch_size=24
epochs = 1000
def get_batch_data(train_data_x,train_data_y,batch_size):
    samples = len(train_data_y)
    n_batch = samples//batch_size
    i=0
    while True:
        i=i%n_batch
        temp_x = train_data_x[i*batch_size:(i+1)*batch_size,:,:,:]
        temp_y = train_data_y[i*batch_size:(i+1)*batch_size,:]
        
#         yield i
        i=i+1
        
        yield temp_x,temp_y

In [155]:
generator = get_batch_data(train_data_x,train_data_y,batch_size)

In [156]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    tf.summary.FileWriter(LOG_DIR,sess.graph)
    num_batches= len(train_data_y)// batch_size
    for i in range(epochs):
        for j in range(num_batches):
            batch_x,batch_y = next(generator)
            feed_dict={x_input:batch_x,y_input:batch_y}
            sess.run(optimizer,feed_dict=feed_dict)
            
            if j % 10 ==0:
                loss_res =  sess.run(loss,feed_dict=feed_dict)
                print(epochs,j,loss_res)

1000 0 1.9524921


KeyboardInterrupt: 